In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import time
import torch
import pathlib
import numpy as np
import pathlib
import json
import numpy as np
import time
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../src/")
from model_bert import BertForSequenceClassification
import torch.nn.utils.prune as prune
from find_global_weight_mask import prune_model, L1UnstructuredInvert



def load_mask_data(experiments_path):
    mask_data = {}
    for task_dir in experiments_path.iterdir():
        mask_data[task_dir.stem] = {}
        for seed_dir in task_dir.iterdir():
            mask = torch.load(seed_dir / "magnitude_mask.p")
            mask_data[task_dir.stem][seed_dir.stem] = mask
    return mask_data
def load_head_data(experiments_path):
    head_data = {}
    for task_dir in experiments_path.iterdir():
        head_data[task_dir.stem] = {}
        for seed_dir in task_dir.iterdir():
            head_mask = np.load(seed_dir / "head_mask.npy")
            head_data[task_dir.stem][seed_dir.stem] = {
                "head_mask": head_mask,
            }
    return head_data
def load_mlp_data(experiments_path):
    mlp_data = {}
    for task_dir in experiments_path.iterdir():
        mlp_data[task_dir.stem] = {}
        for seed_dir in task_dir.iterdir():
            mlp_mask = np.load(seed_dir / "mlp_mask.npy")
            mlp_importance = np.load(seed_dir / "mlp_importance.npy")
            mlp_data[task_dir.stem][seed_dir.stem] = {
                "mlp_mask": mlp_mask,
                "mlp_importance": mlp_importance
            }
    return mlp_data


loading region bounding boxes for computing carbon emissions region, this may take a moment...
 454/454... rate=447.43 Hz, eta=0:00:00, total=0:00:01, wall=13:14 CETT
Done!


In [4]:
experiments_path = pathlib.Path("../masks/heads_mlps")
heads = load_head_data(experiments_path)
experiments_path = pathlib.Path("../masks/heads_mlps")
mlps = load_mlp_data(experiments_path)

models_path = pathlib.Path("../models/finetuned")
global_masks_path = pathlib.Path("../masks/global/")

In [5]:
from torch.nn.utils import prune
def heads_to_prune(model):
    parameters_to_prune = []
    for layer in model.bert.encoder.layer:
        parameters = [
            (layer.attention.self.key, 'weight'),
            (layer.attention.self.key, 'bias'),
            (layer.attention.self.query, 'weight'),
            (layer.attention.self.query, 'bias'),
            (layer.attention.self.value, 'weight'),
            (layer.attention.self.value, 'bias'),
            (layer.attention.output.dense, 'weight'),
            (layer.attention.output.dense, 'bias'),
        ]
        parameters_to_prune.extend(parameters)
    return parameters_to_prune
def mlps_to_prune(model):
    parameters_to_prune = []
    for layer in model.bert.encoder.layer:
        parameters = [
            (layer.intermediate.dense, 'weight'),
            (layer.intermediate.dense, 'bias'),
            (layer.output.dense, 'weight'),
            #(layer.output.dense, 'bias'), Since we don't prune this in importance based pruning.
        ]
        parameters_to_prune.extend(parameters)
    return parameters_to_prune




In [6]:
torch.manual_seed(13)
for task in heads:
    for seed in heads[task]:
        print(task, seed)
        heads_pct = heads[task][seed]["head_mask"].sum() / heads[task][seed]["head_mask"].size
        mlps_pct  = mlps[task][seed]["mlp_mask"].sum() / mlps[task][seed]["mlp_mask"].size
        heads_pct = heads_pct.item()
        mlps_pct = mlps_pct.item()
        print(f"Good Heads: {heads_pct} MLPS: {mlps_pct}")
        model     = BertForSequenceClassification.from_pretrained(str(models_path / task / seed ))
        heads_prune = heads_to_prune(model)
        mlps_prune = mlps_to_prune(model)
        prune.global_unstructured(
            heads_prune,
            pruning_method=prune.L1Unstructured,
            amount=1 - heads_pct,
        )
        prune.global_unstructured(
            mlps_prune,
            pruning_method=prune.L1Unstructured,
            amount=1 - mlps_pct,
        )
        global_mag_masks = {k:v for k, v in model.named_buffers()}
        #torch.save(global_mag_masks, str(global_masks_path/task/seed/"magnitude_mimic.pt"))
        
        print(f"Good Heads: {heads_pct} MLPS: {mlps_pct}")
        model     = BertForSequenceClassification.from_pretrained(str(models_path / task / seed ))
        heads_prune = heads_to_prune(model)
        mlps_prune = mlps_to_prune(model)
        prune.global_unstructured(
            heads_prune,
            pruning_method=L1UnstructuredInvert,
            amount=1 - heads_pct,
        )
        prune.global_unstructured(
            mlps_prune,
            pruning_method=L1UnstructuredInvert,
            amount=1 - mlps_pct,
        )
        global_mag_masks = {k:v for k, v in model.named_buffers()}
        #torch.save(global_mag_masks, str(global_masks_path/task/seed/"bad_mimic.pt"))
        
        
        model     = BertForSequenceClassification.from_pretrained(str(models_path / task / seed ))
        heads_prune = heads_to_prune(model)
        mlps_prune = mlps_to_prune(model)
        prune.global_unstructured(
            heads_prune,
            pruning_method=prune.RandomUnstructured,
            amount=1 - heads_pct,
        )
        prune.global_unstructured(
            mlps_prune,
            pruning_method=prune.RandomUnstructured,
            amount=1 - mlps_pct,
        )
        global_mag_masks = {k:v for k, v in model.named_buffers()}
        #torch.save(global_mag_masks, str(global_masks_path/task/seed/"random_mimic.pt"))


Heads: 0.0 MLPS: 0.0
Heads: 0.0 MLPS: 0.0
Heads: 1.0 MLPS: 0.0
Heads: 0.0 MLPS: 0.0
Heads: 0.0 MLPS: 0.0
Heads: 0.5625 MLPS: 0.75
Heads: 0.6111111111111112 MLPS: 0.6666666666666666
Heads: 0.6597222222222222 MLPS: 0.9166666666666666
Heads: 0.5625 MLPS: 0.75
Heads: 0.7083333333333334 MLPS: 0.75
Heads: 0.6111111111111112 MLPS: 0.5833333333333334
Heads: 0.6597222222222222 MLPS: 0.75
Heads: 0.6597222222222222 MLPS: 0.75
Heads: 0.7083333333333334 MLPS: 0.75
Heads: 0.6597222222222222 MLPS: 0.6666666666666666
Heads: 0.7083333333333334 MLPS: 0.75
Heads: 0.4652777777777778 MLPS: 0.6666666666666666
Heads: 0.6111111111111112 MLPS: 0.75
Heads: 0.4652777777777778 MLPS: 0.75
Heads: 0.4652777777777778 MLPS: 0.6666666666666666
Heads: 0.5138888888888888 MLPS: 0.6666666666666666
Heads: 0.125 MLPS: 0.9166666666666666
Heads: 0.5138888888888888 MLPS: 0.5833333333333334
Heads: 0.8055555555555556 MLPS: 0.9166666666666666
Heads: 0.6111111111111112 MLPS: 0.75
Heads: 0.4166666666666667 MLPS: 0.75
Heads: 0.611111

## Bad masks of equal size as "good" magnitude masks, sampled.

In [57]:


def load_global_mask_data(experiments_path):
    mask_data = {}
    for task_dir in experiments_path.iterdir():
        mask_data[task_dir.stem] = {}
        for seed_dir in task_dir.iterdir():
            mask = torch.load(seed_dir / "magnitude_mask.p")
            mask_data[task_dir.stem][seed_dir.stem] = mask
    return mask_data

global_masks_path = pathlib.Path("../masks/global/")
models_dir = pathlib.Path("../models/finetuned/")
global_masks = load_global_mask_data(global_masks_path)


In [60]:
for task in global_masks:
    for seed in global_masks[task]:
        good_mask = global_masks[task][seed]
        total_masked = sum([(1-v).sum() for v in good_mask.values()])
        total_elements = sum([v.numel() for v in good_mask.values()])
        amount = total_masked/total_elements
        model = BertForSequenceClassification.from_pretrained(str(models_dir / task / seed))
        new_mask = prune_model(model, amount=amount.item(), mode=L1UnstructuredInvert)
        torch.save(new_mask, str(global_masks_path/task/seed/"bad_mask.pt"))

In [10]:

def load_global_mask_data(experiments_path):
    mask_data = {}
    for task_dir in experiments_path.iterdir():
        mask_data[task_dir.stem] = {}
        for seed_dir in task_dir.iterdir():
            mask = torch.load(seed_dir / "magnitude_mask.p")
            mask_data[task_dir.stem][seed_dir.stem] = mask
    return mask_data

def load_global_mask_data_1(experiments_path):
    mask_data = {}
    for task_dir in experiments_path.iterdir():
        mask_data[task_dir.stem] = {}
        for seed_dir in task_dir.iterdir():
            mask = torch.load(seed_dir / "magnitude_mimic.pt")
            mask_data[task_dir.stem][seed_dir.stem] = mask
    return mask_data


In [13]:
do = load_global_mask_data(global_masks_path)
dm = load_global_mask_data_1(global_masks_path)

In [16]:

origgg = []
mimiccc = []
for task in do:
    for seed in do[task]:
        orig = do[task][seed]
        mimic = dm[task][seed]
        pruned_o = sum([(1-v).sum() for v in orig.values()]) / sum([v.numel() for v in orig.values()])
        pruned_m = sum([(1-v).sum() for v in mimic.values()]) / sum([v.numel() for v in mimic.values()])
        #print(f"Pruned o - {pruned_o} Pruned m - {pruned_m}")
        origgg.append(pruned_o)
        mimiccc.append(pruned_m)

In [17]:
sum(origgg) / len(origgg)

tensor(0.4816)

In [18]:
sum(mimiccc) / len(mimiccc)

tensor(0.3901)